In [1]:
import json
import numpy as np
import pandas as pd
import re

from pathlib import Path

In [3]:
df = pd.read_json("./results/multinli_transforms_bertscore_dev_matched.jsonl", lines=True)
df_annotations_1 = pd.read_json("./annotations/raw/annotator_1.json")[['pairID', 'label']]
df_annotations_2 = pd.read_json("./annotations/raw/annotator_2.json")[['pairID', 'label']]

In [4]:
df = df.join(df_annotations_1.set_index('pairID'), on='pairID', rsuffix='_erwin')
df = df.join(df_annotations_2.set_index('pairID'), on='pairID', rsuffix='_ewout')
df = df.dropna()

In [5]:
df_differing = df[df['label'] != df['label_ewout']]

In [8]:
len(df_differing)

62

In [15]:
df_al = pd.read_json("./annotations/gold.json")
total = 0

for index, row in df_differing.iterrows():
    if row['pairID'] in df_al.index.tolist():
        total += 1
print(total)

62


In [31]:
annotation_set = []

for index, row in df_differing.iterrows():
    premise = row['sentence1_transform1'] if row['sentence1_transform1_bertscore'] > row['sentence1_transform2_bertscore'] else row['sentence1_transform2']
    hypothesis = row['sentence2_transform1'] if row['sentence2_transform1_bertscore'] > row['sentence2_transform2_bertscore'] else row['sentence2_transform2']
    annotation_set.append({
        'pairID': row['pairID'],
        'text': f"Premise:\n{premise}\n\nHypothesis:\n{hypothesis}",
        'label': row['gold_label'],
        'orig_prem': row['sentence1'],
        'orig_hyp': row['sentence2'],
        'orig_label': row['gold_label'],
        'label_erwin': row['label'][0],
        'label_ewout': row['label_ewout'][0]
    })
    
annotation_df = pd.DataFrame(annotation_set)

In [33]:
save_path = Path('./results/annotation_consolidation.json')
annotation_df.to_json(save_path, orient="records")